[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GabbyTab/boofun/blob/main/notebooks/lecture6_spectral_concentration.ipynb)

# CS 294-92: Lecture 6 - Spectral Concentration and Low-Degree Learning

**Instructor:** Avishay Tal  
**Scribe & Notebook by:** Gabriel Taboada  
**Reference:** O'Donnell, *Analysis of Boolean Functions*, Chapter 3

---

## Overview

This notebook explores the connection between Boolean function analysis and learning theory:

1. **Spectral Concentration**: When is a function's Fourier weight concentrated on low-degree coefficients?
2. **Decision Trees**: How decision tree depth relates to spectral concentration
3. **PAC Learning**: The LMN Theorem for learning functions with spectral concentration
4. **Fourier Coefficient Estimation**: Using samples to estimate Fourier coefficients

---

In [1]:
# Install/upgrade boofun (required for Colab)
# This ensures you have the latest version with all features
!pip install --upgrade boofun -q

import boofun as bf
print(f"BooFun version: {bf.__version__}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


/Users/gabrieltaboada/dev/Boofun/boofun/src/boofun/core/errormodels.py:21: UserWarning: uncertainties library not available - some error models disabled
  warnings.warn("uncertainties library not available - some error models disabled")


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


BooFun version: 1.1.1


/Users/gabrieltaboada/dev/Boofun/boofun/src/boofun/quantum/__init__.py:22: UserWarning: Qiskit not available - quantum features limited
  warnings.warn("Qiskit not available - quantum features limited")


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import boofun as bf
from boofun.analysis import learning, complexity

import warnings
warnings.filterwarnings('ignore')

## 1. Spectral Concentration

**Definition:** A function $f$ is **$\varepsilon$-concentrated** on degree $\leq k$ if:
$$\mathbf{W}^{>k}[f] := \sum_{|S| > k} \hat{f}(S)^2 \leq \varepsilon$$

This means the "Fourier weight" on high-degree coefficients is small.

### Example: Decision Trees and Spectral Concentration

**Theorem (O'Donnell 3.2):** A depth-$d$ decision tree has all its Fourier mass on degree $\leq d$.

In [3]:
# Demonstrate spectral concentration for different functions
functions = [
    ("Dictator x₀", bf.dictator(4, 0)),    # 4 vars, dictator index 0, depth 1
    ("Majority-5", bf.majority(5)),         # Complex
    ("Parity-4", bf.parity(4)),             # Max degree
    ("Tribes(2,3)", bf.tribes(2, 3)),       # DNF
]

print("Spectral Concentration Analysis")
print("=" * 70)
print(f"{'Function':<15} {'DT depth':>10} {'W≤1':>10} {'W≤2':>10} {'W≤3':>10}")
print("-" * 70)

for name, f in functions:
    dt_depth = complexity.decision_tree_depth(f)
    
    # f.W_leq(k) = sum of f̂(S)² for |S| ≤ k
    conc = [f.W_leq(k) for k in [1, 2, 3]]
    
    print(f"{name:<15} {dt_depth:>10} {conc[0]:>10.4f} {conc[1]:>10.4f} {conc[2]:>10.4f}")

print("\nHigher DT depth → spectral weight spreads to higher degrees")

Spectral Concentration Analysis
Function          DT depth        W≤1        W≤2        W≤3
----------------------------------------------------------------------
Dictator x₀              1     1.0000     1.0000     1.0000
Majority-5               5     0.7031     0.7031     0.8594
Parity-4                 4     0.0000     0.0000     0.0000
Tribes(2,3)              3     0.7500     0.9375     1.0000

Higher DT depth → spectral weight spreads to higher degrees


---
## 2. Fourier Coefficient Estimation

**Lemma (O'Donnell 3.30):** Given $m = O(\log(1/\delta)/\varepsilon^2)$ samples, we can estimate $\hat{f}(S)$ with error $\leq \varepsilon$ with probability $\geq 1 - \delta$.

The empirical estimator is:
$$\tilde{f}(S) = \frac{1}{m} \sum_{i=1}^{m} f(x^{(i)}) \chi_S(x^{(i)})$$

In [4]:
# Demonstrate Fourier coefficient estimation
f = bf.majority(4)
true_coeffs = f.fourier()

# Estimate coefficients with different sample sizes
sample_sizes = [50, 200, 1000]

print("Fourier Coefficient Estimation (Majority-4)")
print("=" * 70)
print(f"{'Subset S':<12} {'True f̂(S)':>12} " + "".join(f"{'m='+str(m):>15}" for m in sample_sizes))
print("-" * 70)

# Show estimation for a few important subsets (LSB=x₀ convention)
subsets_to_show = [0, 1, 2, 4, 8, 3, 5, 15]  # Various subset masks

for s in subsets_to_show:
    # LSB=x₀: bit i of s means variable i is in subset
    subset = [i for i in range(4) if (s >> i) & 1]
    true_val = true_coeffs[s]
    
    row = f"{str(subset):<12} {true_val:>12.4f}"
    
    for m in sample_sizes:
        # Generate samples
        rng = np.random.default_rng(42)
        samples = []
        labels = []
        
        for _ in range(m):
            x = int(rng.integers(0, 16))  # Convert to Python int
            y = 1 - 2 * int(f.evaluate(x))  # Convert to ±1
            samples.append(x)
            labels.append(y)
        
        # Estimate using empirical formula: f̂(S) ≈ (1/m) Σ f(x) χ_S(x)
        est = 0.0
        for x, y in zip(samples, labels):
            chi_s = 1 - 2 * (bin(x & s).count("1") % 2)
            est += y * chi_s
        est /= m
        
        error = abs(est - true_val)
        row += f" {est:>8.4f}±{error:.4f}"
    
    print(row)

Fourier Coefficient Estimation (Majority-4)
Subset S       True f̂(S)            m=50          m=200         m=1000
----------------------------------------------------------------------
[]                 0.3750   0.2400±0.1350   0.3800±0.0050   0.3680±0.0070
[0]                0.3750   0.4800±0.1050   0.4600±0.0850   0.4160±0.0410
[1]                0.3750   0.3600±0.0150   0.3100±0.0650   0.3860±0.0110
[2]                0.3750   0.4000±0.0250   0.3400±0.0350   0.4020±0.0270
[3]                0.3750   0.2000±0.1750   0.2500±0.1250   0.3220±0.0530
[0, 1]            -0.1250  -0.1200±0.0050  -0.1900±0.0650  -0.1540±0.0290
[0, 2]            -0.1250  -0.1600±0.0350  -0.1600±0.0350  -0.1220±0.0030
[0, 1, 2, 3]       0.3750   0.3200±0.0550   0.4000±0.0250   0.3300±0.0450


---
## 3. The LMN Theorem (PAC Learning)

**Theorem (Linial-Mansour-Nisan, 1993):** 
Let $\mathcal{C}$ be a concept class such that every $f \in \mathcal{C}$ is $\varepsilon$-concentrated on Fourier coefficients of degree $\leq k$.
Then $\mathcal{C}$ is $(\varepsilon, \delta)$-PAC learnable in time $\text{poly}(n^k, 1/\varepsilon, \log(1/\delta))$.

### Learning Algorithm

1. Estimate all degree-$\leq k$ Fourier coefficients using samples
2. Construct hypothesis: $h(x) = \text{sgn}\left(\sum_{|S| \leq k} \tilde{f}(S) \chi_S(x)\right)$
3. Fourier concentration bounds the approximation error

In [5]:
# Demonstrate the LMN learning algorithm (simplified version)
from boofun.analysis.learning import estimate_fourier_coefficient

# Create a depth-2 decision tree: (x₀ AND x₁) OR (x₂ AND x₃)
# Truth table: f=1 when (x₀=1 and x₁=1) or (x₂=1 and x₃=1)
tt = []
for i in range(16):
    x0, x1, x2, x3 = [(i >> j) & 1 for j in range(4)]
    val = (x0 and x1) or (x2 and x3)
    tt.append(val)
target = bf.create(tt)

print("LMN Learning Example")
print("=" * 60)
print("Target: (x₀ AND x₁) OR (x₂ AND x₃)")
print(f"Decision tree depth: {complexity.decision_tree_depth(target)}")
print()

# Show spectral concentration: W≤k = sum of f̂(S)² for |S| ≤ k
for k in [1, 2, 3, 4]:
    conc = target.W_leq(k)
    print(f"W≤{k} = {conc:.4f}")

print("\nLMN approach: estimate low-degree coefficients, threshold to classify")

# Estimate degree ≤ 2 coefficients
rng = np.random.default_rng(42)
estimated_coeffs = {}
for s in range(16):
    degree = bin(s).count('1')
    if degree <= 2:
        est, _ = estimate_fourier_coefficient(target, s, num_samples=500, rng=rng)
        if abs(est) > 0.05:
            estimated_coeffs[s] = est

print(f"\nEstimated {len(estimated_coeffs)} non-negligible degree-≤2 coefficients")

# Use estimated coefficients for prediction
def predict(x):
    val = sum(coeff * (1 - 2 * (bin(x & s).count('1') % 2)) 
              for s, coeff in estimated_coeffs.items())
    return 1 if val > 0 else -1

# Test accuracy
correct = sum(1 for x in range(16) 
              if predict(x) == (1 - 2 * int(target.evaluate(x))))

print(f"Learning accuracy: {correct}/16 = {correct/16:.2%}")

LMN Learning Example
Target: (x₀ AND x₁) OR (x₂ AND x₃)
Decision tree depth: 4

W≤1 = 0.5781
W≤2 = 0.9219
W≤3 = 0.9844
W≤4 = 1.0000

LMN approach: estimate low-degree coefficients, threshold to classify

Estimated 11 non-negligible degree-≤2 coefficients
Learning accuracy: 16/16 = 100.00%


---
## 4. The Goldreich-Levin Algorithm

**Problem:** Find all "heavy" Fourier coefficients without enumerating all $2^n$ subsets.

**Theorem (Goldreich-Levin, 1989):** There exists an algorithm that, given oracle access to $f$, finds all $S$ with $|\hat{f}(S)| \geq \tau$ using $O(n/\tau^4)$ queries.

### Key Idea: Self-Correction via Restrictions

For a random subset $T \subseteq [n]$:
$$\mathbf{E}_{T,b}[\hat{f|_{T=b}}(S|_{\bar{T}})] = \hat{f}(S)$$

This allows recursively finding heavy coefficients by restricting variables.

In [6]:
# Demonstrate Goldreich-Levin algorithm
from boofun.analysis.learning import goldreich_levin

# Parity has only one non-zero Fourier coefficient (at the full set)
xor = bf.parity(4)

print("Goldreich-Levin Algorithm Demo")
print("=" * 60)
print("Target: XOR (Parity) on 4 bits")
print(f"Fourier sparsity: only 1 non-zero coefficient")
print()

# True Fourier coefficients
true_coeffs = xor.fourier()

print("True heavy coefficients (|f̂(S)| > 0.1):")
for s in range(len(true_coeffs)):
    if abs(true_coeffs[s]) > 0.1:
        # LSB=x₀: bit i of s means variable i is in subset
        subset = [i for i in range(4) if (s >> i) & 1]
        print(f"  S={subset}: f̂(S) = {true_coeffs[s]:.4f}")

# Use Goldreich-Levin to find heavy coefficients
heavy = goldreich_levin(xor, threshold=0.3)

print(f"\nGoldreich-Levin found {len(heavy)} heavy coefficient(s):")
for s, coeff in heavy:
    subset = [i for i in range(4) if (s >> i) & 1]
    print(f"  S={subset}: estimated f̂(S) = {coeff:.4f}")

Goldreich-Levin Algorithm Demo
Target: XOR (Parity) on 4 bits
Fourier sparsity: only 1 non-zero coefficient

True heavy coefficients (|f̂(S)| > 0.1):
  S=[0, 1, 2, 3]: f̂(S) = 1.0000

Goldreich-Levin found 1 heavy coefficient(s):
  S=[0, 1, 2, 3]: estimated f̂(S) = 1.0000


---
## Summary

### Key Concepts

1. **Spectral Concentration**: Functions with bounded decision tree depth have Fourier weight concentrated on low degrees

2. **LMN Theorem**: If a function class has spectral concentration at degree $k$, it's PAC-learnable in time $n^{O(k)}$

3. **Fourier Coefficient Estimation**: $O(\log(1/\delta)/\varepsilon^2)$ samples suffice to estimate any $\hat{f}(S)$ within $\varepsilon$

4. **Goldreich-Levin**: Find heavy Fourier coefficients efficiently without enumeration

### Corollaries (from lecture notes)

- **Depth-d decision trees**: Learnable in time $n^{O(d)}$
- **Size-s decision trees**: Learnable in time $n^{O(\log s)}$
- **Linear Threshold Functions**: Learnable in time $n^{O(1/\varepsilon^2)}$

### Open Questions

- Can depth-$d$ decision trees be learned in $\text{poly}(n, 2^d)$ time?
- Can $k$-juntas be learned in $\text{poly}(n)$ time for $k = \log n$?
- Efficient learning of small DNF/CNF formulas?